<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #6495ed; margin-bottom:10px;">
  <h4 style="font-size:13px;">📦 <strong>Importar Librerias</strong></h4>
</div>

In [1]:
import sys
import ray
from ray import serve
import mlflow
import pandas as pd
from fastapi import FastAPI
from fastapi import Request
from pydantic import BaseModel, ConfigDict
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse
from starlette.requests import Request
from json import JSONDecodeError
from sklearn.datasets import load_iris
#import requests

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Configuracion del Modelo</strong></h4>
</div>

In [2]:
try:
    class Item(BaseModel):
        model_config = ConfigDict(extra="forbid")
        capitulo_mxf_t0052: str
        cnt_pes_brut_t0051: float
        cnt_totserie_t0051: float
        cod_puertemb_t0051: str
        cod_viatrans_t0051: str
        c390:               float
        c412:               float
        c463:               float
        ddp_ciiu:           str
        dia_ant_dam_altruc: float
        margen_neto:        float
        mto_fob_mfs:        float
        mto_tovaladu_t0051: float
        mto_itf_tot_partic: float
        mto_itf_esf_partic: float
        part_prov_high:     float
        part_prov_medium:   float
        part_prov_outlier:  float
        rat_fob_pneto_mfs:  float
        ruc_agente_dam:     str
        seccion:            str

    app = FastAPI(
        title="Import Predictor API",
        description="Import Pipeline online inference",
    )

    @app.exception_handler(RequestValidationError)
    async def validation_exception_handler(request: Request, exc: RequestValidationError):
        mensajes = [err["msg"] for err in exc.errors()]
        return JSONResponse(
            status_code=422,
            content={
                "cod": 422,
                "msg": "; ".join(mensajes),
                "data": None,
                "errors": exc.errors(),
            },
        )

    @serve.deployment(ray_actor_options={"num_cpus": 0.1})
    @serve.ingress(app)
    class ModelMlFlow:
        def __init__(self, model_uri: str):
            mlflow.set_tracking_uri("http://172.26.56.43:5000")
            try:
                self.model = mlflow.sklearn.load_model(model_uri)
            except Exception as ex:
                print(f"MlFlow: Se produjo el siguiente error : {ex} ")

        @app.post("/")
        async def score(self, item: dict):  # <- evitar pickling de 'Item'
            try:
                payload = item
                df = pd.DataFrame([payload])
                score = float(self.model.predict_proba(df)[0][1])
                return {
                    "cod": 200,
                    "msg": "Score calculated successfully.",
                    "data": {"em_eventprobability": score},
                    "errors": None,
                }
            except Exception as e:
                return {
                    "cod": 500,
                    "msg": "Error calculating score",
                    "data": None,
                    "errors": str(e),
                }

except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex} ")

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Conecta a un cluster de Ray ya iniciado</strong></h4>
</div>

In [3]:
try:
    ray.init(address="ray://172.26.56.65:10001")
    serve.start(detached=True, http_options={"host": "0.0.0.0", "port": 8000})
    print("✅ Conectado a Ray existente.")
except Exception as e:
    sys.exit(f"❌ No se pudo conectar a ray://172.26.56.65:10001 → {e}")

2025-08-19 15:22:41,456	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
2025-08-19 15:22:43,746	WARNING utils.py:1591 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.41.0
    Python: 3.11.11
This process on Ray Client was started with:
    Ray: 2.41.0
    Python: 3.11.6

INFO 2025-08-19 15:22:47,474 serve 2459 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.


✅ Conectado a Ray existente.


<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Desplegar el Modelo</strong></h4>
</div>

In [4]:
MODEL_URI = "models:/sobrevalu-expo-nac/1"   # o "models:/iris-models/Production"
NAME = "sobrevalu-expo-nac"
ROUTE_PREFIX = "/v1/gestionriesgo/evaluacionriesgoaduanero/sobrevalu-expo-nac/score/e/damscore"

graph = ModelMlFlow.bind(MODEL_URI)

try:
    serve.run(graph, name=NAME, route_prefix=ROUTE_PREFIX)
    print(f"✅ Listo en http://172.26.56.65:8000{ROUTE_PREFIX}")
except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex}")
    try:
        serve.delete(NAME)
        print(f"🧹 App '{NAME}' eliminada por fallo de deploy.")
    except Exception as cleanup_ex:
        print(f"⚠️ No se pudo eliminar '{NAME}': {cleanup_ex}")

INFO 2025-08-19 15:22:47,492 serve 2459 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.
WARNING 2025-08-19 15:22:47,493 serve 2459 -- The new client HTTP config differs from the existing one in the following fields: ['host', 'location']. The new HTTP config is ignored.
INFO 2025-08-19 15:22:53,709 serve 2459 -- Application 'sobrevalu-expo-nac' is ready at http://0.0.0.0:8000/v1/gestionriesgo/evaluacionriesgoaduanero/sobrevalu-expo-nac/score/e/damscore.
INFO 2025-08-19 15:22:53,711 serve 2459 -- Deployed app 'sobrevalu-expo-nac' successfully.


✅ Listo en http://172.26.56.65:8000/v1/gestionriesgo/evaluacionriesgoaduanero/sobrevalu-expo-nac/score/e/damscore


In [5]:
ray.shutdown()